# References

The RBF Neural Network is based on the implementation of [this github repo](https://github.com/darecophoenixx/wordroid.sblo.jp/tree/master/lib/keras_ex/gkernel)


Also, to train the MNIST dataset, [his example of digit recognizer](https://www.kaggle.com/wordroid/keras-rbf-layer-quick-start) is heavily used.




Links (if you cannot open):
* https://github.com/darecophoenixx/wordroid.sblo.jp/tree/master/lib/keras_ex/gkernel
* https://www.kaggle.com/wordroid/keras-rbf-layer-quick-start

# Install the Library

In [1]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

  Cloning https://github.com/darecophoenixx/wordroid.sblo.jp to /tmp/pip-req-build-b3d_tneo
  Stored in directory: /tmp/pip-ephem-wheel-cache-evnaf8uq/wheels/3d/6d/24/44e7d6dc47b39e74772a04beb27433ba2ebed90876394cda60
Successfully built wordroid.sblo.jp
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Keras and tf versions

In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [3]:
import tensorflow as tf
tf.__version__

'1.13.1'

# Other libraries that are used

In [4]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

# Get MNIST data and normalize

In [5]:
(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
X = X.reshape((X.shape[0], -1))
X_sc = X / 256.0
X_sc.shape
y_cat = to_categorical(y)

In [7]:
X_test = X_test.reshape((X_test.shape[0], -1))
X_test_sc = X_test / 256.0


 # Choose landmarks 

In [8]:
'''
chose landmarks from sample
pick 10 data each digit
'''
np.random.seed(0)
num_lm0 = 100
num_lm = num_lm0 * 10
init_list = []
for ii in range(10):
    init_wgt0 = X_sc[y==ii]
    init_wgt0 = init_wgt0[np.random.choice(range(init_wgt0.shape[0]), size=num_lm0, replace=False)] + \
                np.random.normal(scale=0.01, size=num_lm0*784).reshape(num_lm0, 784)
    init_list.append(init_wgt0)
init_wgt = np.vstack(init_list)
init_wgt = init_wgt[np.random.permutation(range(init_wgt.shape[0]))]
init_wgt.shape

(1000, 784)

# Construct the model

In [9]:


np.random.seed(0)

inp = Input(shape=(28*28,), name='inp')
oup = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp)
oup = Dense(10, activation='softmax')(oup)
model = Model(inp, oup)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 784)               0         
_________________________________________________________________
gkernel1 (GaussianKernel)    (None, 1000)              784000    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10010     
Total params: 794,010
Trainable params: 794,010
Non-trainable params: 0
_________________________________________________________________


# Train the model

In [10]:
model.fit(X_sc, y_cat, verbose=1,batch_size=500,epochs=250)

Instructions for updating:
Use tf.cast instead.
Epoch 1/250
60000/60000 [==============================] - 4s 75us/step - loss: 1.8494 - acc: 0.5292
Epoch 2/250
60000/60000 [==============================] - 4s 69us/step - loss: 1.1754 - acc: 0.7643
Epoch 3/250
60000/60000 [==============================] - 4s 69us/step - loss: 0.8726 - acc: 0.8070
Epoch 4/250
60000/60000 [==============================] - 4s 71us/step - loss: 0.7117 - acc: 0.8394
Epoch 5/250
60000/60000 [==============================] - 4s 69us/step - loss: 0.6176 - acc: 0.8561
Epoch 6/250
60000/60000 [==============================] - 4s 69us/step - loss: 0.5559 - acc: 0.8649
Epoch 7/250
60000/60000 [==============================] - 4s 70us/step - loss: 0.5081 - acc: 0.8752
Epoch 8/250
60000/60000 [==============================] - 4s 70us/step - loss: 0.4759 - acc: 0.8809
Epoch 9/250
60000/60000 [==============================] - 4s 69us/step - loss: 0.4493 - acc: 0.8861
Epoch 10/250
60000/60000 [=================

# Predict and Analyse

In [11]:
y_pred_test = model.predict(X_test_sc)
y_pred_train = model.predict(X_sc)

In [12]:
print("Training accuracy: ",accuracy_score(y,np.argmax(y_pred_train,axis=1)))
print("Test accuracy: ",accuracy_score(y_test,np.argmax(y_pred_test,axis=1)))

Training accuracy:  0.93305
Test accuracy:  0.9297


In [13]:
print("Conf matrix train:\n ",confusion_matrix(y,np.argmax(y_pred_train,axis=1)))
print("Conf matrix test:\n ",confusion_matrix(y_test,np.argmax(y_pred_test,axis=1)))

Conf matrix train:
  [[5759    1    9    9   12   37   37    7   48    4]
 [   1 6620   16   20    6   21    3    7   38   10]
 [  33  123 5236  102   81   24   79   79  172   29]
 [  20   50   69 5540    5  196   18   48  124   61]
 [  11   35   16    4 5480    6   53    6   34  197]
 [  48   31   13  106   48 4946   74   12   97   46]
 [  33   26   16    1   46   76 5685    3   31    1]
 [  16   45   42   16   50   10    2 5879   16  189]
 [  30  137   22   94   24   97   33   11 5342   61]
 [  24   42    8   65  127   29    2  109   47 5496]]
Conf matrix test:
  [[ 964    0    1    2    0    3    5    2    3    0]
 [   0 1120    2    2    0    1    4    2    4    0]
 [   7   15  899   20    9    3   15   11   45    8]
 [   3    2    8  929    1   23    2   11   19   12]
 [   1    3    2    1  918    0   11    2    6   38]
 [   9    4    0   26   10  790   16    5   24    8]
 [  10    3    4    0    9   16  910    2    4    0]
 [   1   15   14    7    9    2    0  944    3   33]
 [  

In [14]:
print("PRFS train:\n",precision_recall_fscore_support(y,np.argmax(y_pred_train,axis=1)))
print("PRFS test:\n",precision_recall_fscore_support(y_test,np.argmax(y_pred_test,axis=1)))

PRFS train:
 (array([0.96384937, 0.93108298, 0.96126308, 0.92999832, 0.93213131,
       0.90885704, 0.949716  , 0.95422821, 0.89796604, 0.90187069]), array([0.97231133, 0.98190448, 0.8788184 , 0.90360463, 0.93803492,
       0.91237779, 0.96062859, 0.93838787, 0.91300632, 0.92385275]), array([0.96806186, 0.95581865, 0.91819377, 0.91661152, 0.9350738 ,
       0.91061401, 0.95514113, 0.94624175, 0.90542373, 0.91272939]), array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))
PRFS test:
 (array([0.95445545, 0.94834886, 0.96252677, 0.92071358, 0.92540323,
       0.91435185, 0.93621399, 0.94779116, 0.88789683, 0.89748549]), array([0.98367347, 0.98678414, 0.87112403, 0.91980198, 0.93482688,
       0.88565022, 0.94989562, 0.91828794, 0.91889117, 0.9197225 ]), array([0.96884422, 0.9671848 , 0.9145473 , 0.92025755, 0.93009119,
       0.89977221, 0.94300518, 0.93280632, 0.90312815, 0.90846794]), array([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009]))


# Save and Load the Model

In [15]:
model.save_weights("rbfnn.h5")

In [16]:
inp2 = Input(shape=(28*28,), name='inp')
oup2 = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp2)
oup2 = Dense(10, activation='softmax')(oup2)
model2 = Model(inp2, oup2)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model2.load_weights("rbfnn.h5")

In [18]:
y_pred_test = model2.predict(X_test_sc)
y_pred_train = model2.predict(X_sc)

In [19]:
print("Training accuracy: ",accuracy_score(y,np.argmax(y_pred_train,axis=1)))
print("Test accuracy: ",accuracy_score(y_test,np.argmax(y_pred_test,axis=1)))

Training accuracy:  0.93305
Test accuracy:  0.9297
